# 미국의 기준금리 인상 및 은행 파산이 국내 금융지주사에 미치는 영향

공변량 후보: 미국, 한국(환율, 경제 성장률, GDP, 금리, 주식지수)

환율 USD, KRW, USD/KRW  V

주식 지수: KOSPI, KRX100, DJI 

무역 거래량? 외교지수?

## 데이터 준비

In [1]:
import FinanceDataReader as fdr
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np
from datetime import datetime, timedelta
from causalimpact import CausalImpact
import warnings
warnings.filterwarnings('ignore')

In [2]:
# KS11 (KOSPI 지수), 2022년~현재
df = fdr.DataReader('KS11', '2022')
df = pd.DataFrame(df)
x = pd.DataFrame(df['Close'])
df = fdr.DataReader('USD/KRW', '2022')
x = pd.merge(x, df['Close'], how='left', on='Date')
df = fdr.DataReader('S&P500', '2022')
x = pd.merge(x, df['Close'], how='left', on='Date')
x = x.rename(columns={'Close_x':'KOSPI', 'Close_y':'USD/KRW', 'Close':'SP500'})
x

,KOSPI,USD/KRW,SP500
Date,,,
2022-01-04,2989.239990,1194.680054,4793.540039
2022-01-05,2953.969971,1196.500000,4700.580078
2022-01-06,2920.530029,1199.250000,4696.049805
2022-01-07,2954.889893,1205.780029,4677.029785
2022-01-10,2926.719971,1196.880005,4670.290039
...,...,...,...
2023-11-29,2519.810059,1287.530029,4550.580078
2023-11-30,2535.290039,1288.359985,4567.799805
2023-12-01,2505.010010,1296.660034,4594.629883


## 한국 금융지주사

### 시중은행금융지주

In [3]:
df1 = fdr.DataReader('105560', '2022') #KB
df1 = pd.DataFrame(df1)
df1

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,54900,55700,54800,55300,635124,0.005455
2022-01-04,55800,56400,55600,56400,927067,0.019892
2022-01-05,56400,57100,55800,56600,1429634,0.003546
2022-01-06,56600,56900,55500,55800,988985,-0.014134
2022-01-07,56000,56100,54800,55700,1193940,-0.001792
...,...,...,...,...,...,...
2023-11-29,51800,52200,51500,51700,1022798,-0.013359
2023-11-30,51700,52600,51500,52400,1465252,0.013540
2023-12-01,52600,52800,51900,51900,453496,-0.009542


In [4]:
df2 = fdr.DataReader('316140', '2022') #우리
df2 = pd.DataFrame(df2)
df2

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,12600,12950,12600,12800,1498991,0.007874
2022-01-04,12800,13000,12750,13000,2130795,0.015625
2022-01-05,13150,13200,12950,13050,2385685,0.003846
2022-01-06,13050,13150,12800,12850,1716327,-0.015326
2022-01-07,12950,13100,12850,13100,1922826,0.019455
...,...,...,...,...,...,...
2023-11-29,12830,12920,12780,12800,1247453,-0.009288
2023-11-30,12800,13020,12780,13020,3483732,0.017187
2023-12-01,12950,13010,12900,12910,854419,-0.008449


In [5]:
df3 = fdr.DataReader('086790', '2022') #하나
df3 = pd.DataFrame(df3)
df3

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,42050,42600,42000,42350,693817,0.007134
2022-01-04,42450,42800,42250,42700,664584,0.008264
2022-01-05,42950,43350,42500,43000,1487051,0.007026
2022-01-06,42850,43250,42650,43200,1051238,0.004651
2022-01-07,43200,43500,42400,43450,1031482,0.005787
...,...,...,...,...,...,...
2023-11-29,41350,41700,41100,41250,713062,-0.008413
2023-11-30,41100,41900,40900,41650,1473897,0.009697
2023-12-01,42000,42100,41450,41850,679945,0.004802


In [6]:
df4 = fdr.DataReader('055550', '2022') #신한
df4 = pd.DataFrame(df4)
df4

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,36700,37450,36700,37250,1090787,0.012228
2022-01-04,37300,37600,37200,37600,1128934,0.009396
2022-01-05,37800,37950,37300,37500,1473096,-0.002660
2022-01-06,37650,37850,37150,37300,1232985,-0.005333
2022-01-07,37300,37650,36900,37650,1126929,0.009383
...,...,...,...,...,...,...
2023-11-29,36600,36950,36200,36300,726167,-0.018919
2023-11-30,36200,36900,36050,36900,1461987,0.016529
2023-12-01,36900,36950,36300,36400,627672,-0.013550


### 지방은행금융지주

In [7]:
df5 = fdr.DataReader('139130', '2022') #DGB
df5 = pd.DataFrame(df5)
df5

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,9350,9400,9300,9330,226384,-0.005330
2022-01-04,9370,9390,9300,9390,233945,0.006431
2022-01-05,9450,9530,9420,9480,342435,0.009585
2022-01-06,9430,9510,9410,9440,217857,-0.004219
2022-01-07,9450,9490,9370,9490,364404,0.005297
...,...,...,...,...,...,...
2023-11-29,8400,8470,8350,8410,503880,-0.003555
2023-11-30,8390,8540,8340,8500,913125,0.010702
2023-12-01,8500,8530,8450,8480,279838,-0.002353


In [8]:
df6 = fdr.DataReader('138930', '2022') #BNK
df6 = pd.DataFrame(df6)
df6

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,8360,8450,8340,8390,826024,-0.001190
2022-01-04,8420,8470,8350,8470,903737,0.009535
2022-01-05,8530,8590,8470,8550,1909652,0.009445
2022-01-06,8510,8600,8430,8480,1045894,-0.008187
2022-01-07,8510,8570,8410,8560,1244991,0.009434
...,...,...,...,...,...,...
2023-11-29,7170,7220,7140,7160,767912,-0.004172
2023-11-30,7160,7190,7120,7170,725675,0.001397
2023-12-01,7170,7190,7150,7160,331116,-0.001395


In [9]:
df7 = fdr.DataReader('175330', '2022') #JB
df7 = pd.DataFrame(df7)
df7

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,8310,8350,8240,8260,284897,-0.010778
2022-01-04,8280,8370,8250,8370,230459,0.013317
2022-01-05,8400,8450,8360,8390,297356,0.002389
2022-01-06,8390,8440,8320,8340,160870,-0.005959
2022-01-07,8400,8430,8320,8340,230019,0.000000
...,...,...,...,...,...,...
2023-11-29,10750,10880,10720,10790,233653,-0.002773
2023-11-30,10720,10800,10360,10390,1040579,-0.037071
2023-12-01,10390,10480,10280,10420,343922,0.002887


### 비은행금융지주

In [10]:
df8 = fdr.DataReader('138040', '2022') #메리츠
df8 = pd.DataFrame(df8)
df8

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,43550,46300,43000,45500,355882,0.034091
2022-01-04,45500,50000,45500,48000,695595,0.054945
2022-01-05,47800,49450,46900,47000,504312,-0.020833
2022-01-06,46550,48300,46000,46800,389948,-0.004255
2022-01-07,46700,50700,46500,49300,621791,0.053419
...,...,...,...,...,...,...
2023-11-29,54700,54900,53900,54200,350076,-0.014545
2023-11-30,54300,55600,54000,55500,523985,0.023985
2023-12-01,55500,56000,55100,55700,267342,0.003604


In [11]:
df9 = fdr.DataReader('071050', '2022') #한국금융
df9 = pd.DataFrame(df9)
df9

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,80700,80900,79700,79800,107135,-0.011152
2022-01-04,79400,79900,79000,79700,129143,-0.001253
2022-01-05,79000,79900,78100,78900,208624,-0.010038
2022-01-06,78000,78800,76700,76800,282331,-0.026616
2022-01-07,76600,79000,76600,78800,178641,0.026042
...,...,...,...,...,...,...
2023-11-29,58400,58800,57900,58200,40440,-0.011885
2023-11-30,58600,59900,57500,59900,171083,0.029210
2023-12-01,59400,60200,58700,58700,65779,-0.020033


### 은행

In [12]:
df10 = fdr.DataReader('024110', '2022') #기업
df10 = pd.DataFrame(df10)
df10

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,10300,10400,10250,10350,1125994,0.004854
2022-01-04,10400,10400,10300,10400,902538,0.004831
2022-01-05,10450,10500,10400,10450,1607535,0.004808
2022-01-06,10400,10450,10300,10350,1188128,-0.009569
2022-01-07,10350,10400,10300,10350,1371527,0.000000
...,...,...,...,...,...,...
2023-11-29,11720,11760,11680,11720,742385,0.000000
2023-11-30,11720,11830,11700,11830,1744179,0.009386
2023-12-01,11830,11870,11780,11870,917500,0.003381


In [13]:
df11 = fdr.DataReader('006220', '2022') #제주
df11 = pd.DataFrame(df11)
df11

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,6110,6190,5920,6070,251919,-0.009788
2022-01-04,6090,6100,5980,6090,183725,0.003295
2022-01-05,6200,6350,6090,6260,449399,0.027915
2022-01-06,6540,6770,6140,6150,1098925,-0.017572
2022-01-07,6200,6530,6180,6390,618650,0.039024
...,...,...,...,...,...,...
2023-11-29,8120,8150,8040,8080,194459,-0.012225
2023-11-30,8060,8160,7910,8120,187927,0.004950
2023-12-01,8150,8240,8070,8080,151935,-0.004926


In [14]:
df12 = fdr.DataReader('323410', '2022') #카카오
df12 = pd.DataFrame(df12)
df12

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,59000,59800,58900,59100,716944,0.001695
2022-01-04,59100,59200,57200,57300,1221416,-0.030457
2022-01-05,57400,57400,55000,55600,2068038,-0.029668
2022-01-06,55200,56700,54600,56000,2263007,0.007194
2022-01-07,55600,56200,55000,55000,1180315,-0.017857
...,...,...,...,...,...,...
2023-11-29,26300,26350,25850,26000,723505,-0.011407
2023-11-30,26000,26200,25600,26200,776201,0.007692
2023-12-01,26100,26150,25450,25450,641374,-0.028626


## 미국 금융지주사

### 시중은행 금융지주사

In [15]:
adf1 = fdr.DataReader('BAC', '2022-01-01') #BAC (Bank of America)
adf1 = pd.DataFrame(adf1)
adf1 

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,45.090000,46.490002,44.980000,46.180000,43.766479,58587900
2022-01-04,47.060001,48.430000,47.040001,47.990002,45.481880,74606700
2022-01-05,48.139999,48.369999,47.160000,47.180000,44.714211,57791600
2022-01-06,47.980000,48.160000,47.189999,48.130001,45.614559,52040900
2022-01-07,48.150002,49.310001,48.060001,49.180000,46.609688,59082600
...,...,...,...,...,...,...
2023-11-28,29.530001,29.650000,29.320000,29.530001,29.296177,29705800
2023-11-29,29.879999,30.610001,29.809999,30.309999,30.070000,57954900
2023-11-30,30.209999,30.549999,30.040001,30.490000,30.490000,52547700


In [16]:
adf2 = fdr.DataReader('C', '2022-01-01') #C (Citigroup)
adf2 = pd.DataFrame(adf2)
adf2

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,61.180000,63.610001,61.110001,63.099998,57.983276,30508500
2022-01-04,64.059998,64.730003,63.540001,63.590000,58.433552,30303600
2022-01-05,64.099998,64.410004,62.779999,62.849998,57.753551,25149100
2022-01-06,64.220001,65.309998,63.509998,64.910004,59.646523,33334200
2022-01-07,65.180000,66.550003,65.180000,65.779999,60.445961,26548200
...,...,...,...,...,...,...
2023-11-28,45.029999,45.040001,44.560001,44.860001,44.860001,14064700
2023-11-29,45.230000,46.029999,45.160000,45.750000,45.750000,14027600
2023-11-30,45.790001,46.169998,45.580002,46.099998,46.099998,16249000


In [17]:
adf3 = fdr.DataReader('WFC', '2022-01-01') #WFC (Wells Fargo)
adf3 = pd.DataFrame(adf3)
adf3

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,48.930000,50.810001,48.919998,50.730000,48.061909,38978100
2022-01-04,51.580002,53.150002,51.520000,52.750000,49.975666,43855100
2022-01-05,52.919998,53.139999,52.000000,52.290001,49.539856,35072600
2022-01-06,53.150002,53.669998,52.660000,53.630001,50.809380,29720800
2022-01-07,53.860001,55.189999,53.799999,54.770000,51.889431,35149300
...,...,...,...,...,...,...
2023-11-28,42.970001,43.450001,42.790001,43.369999,43.369999,15332900
2023-11-29,43.689999,44.139999,43.630001,43.779999,43.779999,16746000
2023-11-30,43.810001,44.750000,43.480000,44.590000,44.590000,23247500


In [18]:
adf4 = fdr.DataReader('JPM', '2022-01-01') #JPM (JP Morgan)
adf4 = pd.DataFrame(adf4)
adf4

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,159.860001,162.639999,159.509995,161.699997,152.215118,13120900
2022-01-04,164.309998,168.580002,164.229996,167.830002,157.985550,20195800
2022-01-05,167.820007,168.360001,163.729996,163.779999,155.097260,17539400
2022-01-06,166.910004,167.369995,163.869995,165.520004,156.745010,14047500
2022-01-07,165.669998,167.529999,165.059998,167.160004,158.298065,13913300
...,...,...,...,...,...,...
2023-11-28,153.220001,153.869995,152.720001,153.539993,153.539993,6582700
2023-11-29,154.169998,155.630005,154.000000,154.320007,154.320007,9126100
2023-11-30,154.910004,156.130005,154.380005,156.080002,156.080002,9869800


In [19]:
adf5 = fdr.DataReader('GS', '2022-01-01') #GS (Goldman Sachs)
adf5 = pd.DataFrame(adf5)
adf5

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,389.000000,399.299988,387.339996,395.329987,373.335663,3334300
2022-01-04,402.230011,410.279999,401.179993,407.480011,384.809753,3916400
2022-01-05,409.799988,412.660004,397.890015,398.630005,376.452118,3469600
2022-01-06,401.000000,401.200012,386.429993,396.929993,374.846680,2960900
2022-01-07,397.670013,400.549988,394.529999,397.510010,375.394409,2360700
...,...,...,...,...,...,...
2023-11-28,337.380005,338.399994,334.869995,337.649994,334.899994,1419000
2023-11-29,337.000000,343.529999,336.510010,340.260010,340.260010,2038100
2023-11-30,341.679993,344.760010,340.089996,341.540009,341.540009,1903600


### 지방은행 금융지주사

In [20]:
adf6 = fdr.DataReader('USB', '2022-01-01') #USB (US Bancorp)
adf6 = pd.DataFrame(adf6)
adf6

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,56.080002,57.400002,56.080002,57.270000,52.658733,5430900
2022-01-04,58.110001,59.080002,57.910000,58.770000,54.037964,7552600
2022-01-05,59.189999,59.290001,58.400002,58.459999,53.752914,6527200
2022-01-06,59.529999,60.070000,58.939999,60.049999,55.214890,5783200
2022-01-07,59.900002,61.169998,59.680000,61.000000,56.088398,7040100
...,...,...,...,...,...,...
2023-11-28,36.669998,37.139999,36.490002,37.040001,37.040001,9665900
2023-11-29,37.349998,38.099998,37.259998,37.790001,37.790001,7772300
2023-11-30,38.000000,38.279999,37.669998,38.119999,38.119999,10283300


In [21]:
adf7 = fdr.DataReader('PNC', '2022-01-01') #PNC (PNC Financial Services)
adf7 = pd.DataFrame(adf7)
adf7

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,204.600006,208.520004,203.009995,208.220001,192.304550,2199500
2022-01-04,210.179993,218.839996,210.179993,215.889999,199.388290,2491200
2022-01-05,216.740005,218.360001,212.229996,212.389999,196.155807,2771900
2022-01-06,218.339996,220.690002,214.750000,220.570007,203.710571,2970700
2022-01-07,220.520004,224.520004,220.070007,223.729996,206.629028,2173600
...,...,...,...,...,...,...
2023-11-28,128.600006,129.729996,127.830002,129.589996,129.589996,1592700
2023-11-29,130.470001,133.169998,130.100006,132.380005,132.380005,2492800
2023-11-30,132.619995,134.699997,132.110001,133.960007,133.960007,3265200


In [22]:
adf8 = fdr.DataReader('TFC', '2022-01-01') #TFC (Truist Financial)
adf8 = pd.DataFrame(adf8)
adf8

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,59.330002,60.470001,59.299999,60.400002,54.487373,5677600
2022-01-04,61.189999,62.860001,61.139999,62.349998,56.246479,5741500
2022-01-05,62.480000,63.110001,61.619999,61.669998,55.633045,5194800
2022-01-06,62.790001,63.990002,62.150002,63.980000,57.716919,7388100
2022-01-07,63.720001,65.190002,63.410000,65.000000,58.637062,7178800
...,...,...,...,...,...,...
2023-11-28,31.440001,31.700001,31.230000,31.559999,31.559999,6113600
2023-11-29,31.910000,32.709999,31.799999,32.130001,32.130001,9377100
2023-11-30,32.340000,32.430000,31.990000,32.139999,32.139999,12301000


In [23]:
adf9 = fdr.DataReader('FITB', '2022-01-01') #FITB (Fifth Third Bancorp)
adf9 = pd.DataFrame(adf9)
adf9

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,43.990002,44.740002,43.990002,44.669998,41.459862,4020900
2022-01-04,45.070000,46.779999,45.070000,46.570000,43.223328,5534000
2022-01-05,46.880001,47.310001,46.310001,46.430000,43.093384,5632100
2022-01-06,47.320000,48.480000,47.270000,48.369999,44.893970,5831700
2022-01-07,48.259998,48.950001,47.950001,48.799999,45.293068,5191200
...,...,...,...,...,...,...
2023-11-28,27.150000,27.469999,26.889999,27.430000,27.430000,3930200
2023-11-29,27.680000,28.559999,27.650000,28.410000,28.410000,5331300
2023-11-30,28.520000,29.160000,28.290001,28.950001,28.950001,7949000


### 비은행 금융지주사

In [24]:
adf10 = fdr.DataReader('STT', '2022-01-01') #STT (State Street)
adf10 = pd.DataFrame(adf10)
adf10

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,93.900002,94.940002,93.570000,94.449997,88.875969,1614600
2022-01-04,95.769997,99.190002,95.769997,98.320000,92.517586,2367300
2022-01-05,98.900002,99.989998,97.070000,97.150002,91.416618,2907200
2022-01-06,98.440002,99.769997,97.169998,98.919998,93.082169,2494600
2022-01-07,98.650002,102.320000,98.470001,101.660004,95.660469,4315700
...,...,...,...,...,...,...
2023-11-28,69.889999,70.430000,69.580002,70.110001,70.110001,1732700
2023-11-29,70.650002,71.989998,70.500000,71.739998,71.739998,1653500
2023-11-30,71.910004,72.940002,71.400002,72.820000,72.820000,3887100


In [25]:
adf11 = fdr.DataReader('SCHW', '2022-01-01') #SCHW (Charles Schwab)
adf11 = pd.DataFrame(adf11)
adf11

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,84.570000,85.860001,84.139999,85.760002,83.401245,6739700
2022-01-04,87.029999,90.849998,86.809998,89.959999,87.485733,12870700
2022-01-05,90.000000,90.660004,89.300003,89.309998,86.853600,9568800
2022-01-06,90.910004,91.500000,89.449997,90.739998,88.244263,7941200
2022-01-07,90.550003,92.320000,90.099998,91.739998,89.216766,7344300
...,...,...,...,...,...,...
2023-11-28,55.970001,56.599998,55.590000,55.810001,55.810001,7866000
2023-11-29,56.000000,59.930000,56.000000,59.610001,59.610001,15656600
2023-11-30,59.660000,61.799999,59.610001,61.320000,61.320000,20200300


In [26]:
adf12 = fdr.DataReader('MS', '2022-01-01') #MS (Morgan Stanley)
adf12 = pd.DataFrame(adf12)
adf12

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,99.330002,101.400002,99.300003,100.190002,93.174469,8481900
2022-01-04,102.000000,104.680000,101.980003,104.260002,96.959473,8967000
2022-01-05,103.699997,105.029999,101.570000,101.680000,94.560135,10288800
2022-01-06,104.120003,104.209999,100.620003,103.419998,96.178284,10059200
2022-01-07,103.510002,104.529999,102.930000,104.050003,96.764175,6128400
...,...,...,...,...,...,...
2023-11-28,77.360001,77.449997,76.330002,76.879997,76.879997,9197900
2023-11-29,77.480003,79.529999,77.339996,78.550003,78.550003,7420700
2023-11-30,78.769997,79.480003,78.209999,79.339996,79.339996,8480500


In [27]:
adf13 = fdr.DataReader('BLK', '2022-01-01') #BLK (BlackRock)
adf13 = pd.DataFrame(adf13)
adf13

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,918.340027,922.070007,905.429993,911.729980,867.387512,470800
2022-01-04,917.719971,927.479980,916.559998,917.219971,872.610535,495800
2022-01-05,918.150024,925.500000,890.830017,891.770020,848.398315,573300
2022-01-06,893.359985,898.179993,884.030029,895.489990,851.937439,637000
2022-01-07,895.270020,896.630005,884.000000,892.270020,848.874023,643300
...,...,...,...,...,...,...
2023-11-28,725.859985,738.830017,725.859985,736.760010,736.760010,610200
2023-11-29,742.280029,751.270020,742.280029,747.299988,747.299988,750000
2023-11-30,748.159973,751.890015,744.690002,751.229980,751.229980,961300


In [28]:
adf14 = fdr.DataReader('V', '2022-01-01') #V (VanGuard)
adf14 = pd.DataFrame(adf14)
adf14

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,217.520004,222.059998,217.009995,221.429993,218.036133,7694500
2022-01-04,222.000000,226.789993,221.809998,222.460007,219.050339,10629300
2022-01-05,222.639999,225.270004,219.919998,220.000000,216.628067,9594600
2022-01-06,217.800003,221.910004,217.550003,219.750000,216.381866,7877500
2022-01-07,217.309998,220.199997,215.639999,216.960007,213.634644,10702100
...,...,...,...,...,...,...
2023-11-28,253.850006,254.149994,251.610001,252.940002,252.940002,5234900
2023-11-29,253.919998,256.070007,253.520004,254.229996,254.229996,4955100
2023-11-30,254.250000,256.769989,253.169998,256.679993,256.679993,6874700


In [29]:
adf10['Close']

Date
2022-01-03     94.449997
2022-01-04     98.320000
2022-01-05     97.150002
2022-01-06     98.919998
2022-01-07    101.660004
                 ...    
2023-11-28     70.110001
2023-11-29     71.739998
2023-11-30     72.820000
2023-12-01     73.919998
2023-12-04     74.099998
Name: Close, Length: 483, dtype: float64

## 데이터 전처리

### 날짜 분리

In [30]:
start_date = "2022-01-04"
end_date = "2023-11-27"

In [31]:
dfs = []  # DataFrame들을 저장할 리스트

for i in range(1, 13):
    df = eval('df{}'.format(i))  # 직접 DataFrame에 접근
    dfs.append(df['Close'])  # 'Close' 열을 리스트에 추가

# DataFrame들을 병합
merged_df = dfs[0]  # 첫 번째 DataFrame으로 시작
for i in range(1, len(dfs)):
    merged_df = pd.merge(merged_df, dfs[i], on='Date', how='inner')

# 병합된 DataFrame 출력
merged_df

,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y
Date,,,,,,,,,,,,
2022-01-03,55300,12800,42350,37250,9330,8390,8260,45500,79800,10350,6070,59100
2022-01-04,56400,13000,42700,37600,9390,8470,8370,48000,79700,10400,6090,57300
2022-01-05,56600,13050,43000,37500,9480,8550,8390,47000,78900,10450,6260,55600
2022-01-06,55800,12850,43200,37300,9440,8480,8340,46800,76800,10350,6150,56000
2022-01-07,55700,13100,43450,37650,9490,8560,8340,49300,78800,10350,6390,55000
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,51700,12800,41250,36300,8410,7160,10790,54200,58200,11720,8080,26000
2023-11-30,52400,13020,41650,36900,8500,7170,10390,55500,59900,11830,8120,26200
2023-12-01,51900,12910,41850,36400,8480,7160,10420,55700,58700,11870,8080,25450


In [32]:
adfs = []  # DataFrame들을 저장할 리스트

for i in range(1, 15):
    adf = eval('adf{}'.format(i))  # 직접 DataFrame에 접근
    adfs.append(adf['Close'])  # 'Close' 열을 리스트에 추가

# DataFrame들을 병합
amerged_df = adfs[0]  # 첫 번째 DataFrame으로 시작
for i in range(1, len(adfs)):
    amerged_df = pd.merge(amerged_df, adfs[i], on='Date', how='inner')

# 병합된 DataFrame 출력
amerged_df

,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y,Close_x,Close_y
Date,,,,,,,,,,,,,,
2022-01-03,46.180000,63.099998,50.730000,161.699997,395.329987,57.270000,208.220001,60.400002,44.669998,94.449997,85.760002,100.190002,911.729980,221.429993
2022-01-04,47.990002,63.590000,52.750000,167.830002,407.480011,58.770000,215.889999,62.349998,46.570000,98.320000,89.959999,104.260002,917.219971,222.460007
2022-01-05,47.180000,62.849998,52.290001,163.779999,398.630005,58.459999,212.389999,61.669998,46.430000,97.150002,89.309998,101.680000,891.770020,220.000000
2022-01-06,48.130001,64.910004,53.630001,165.520004,396.929993,60.049999,220.570007,63.980000,48.369999,98.919998,90.739998,103.419998,895.489990,219.750000
2022-01-07,49.180000,65.779999,54.770000,167.160004,397.510010,61.000000,223.729996,65.000000,48.799999,101.660004,91.739998,104.050003,892.270020,216.960007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-28,29.530001,44.860001,43.369999,153.539993,337.649994,37.040001,129.589996,31.559999,27.430000,70.110001,55.810001,76.879997,736.760010,252.940002
2023-11-29,30.309999,45.750000,43.779999,154.320007,340.260010,37.790001,132.380005,32.130001,28.410000,71.739998,59.610001,78.550003,747.299988,254.229996
2023-11-30,30.490000,46.099998,44.590000,156.080002,341.540009,38.119999,133.960007,32.139999,28.950001,72.820000,61.320000,79.339996,751.229980,256.679993


In [33]:
# 열 이름에 숫자를 붙여 변경
new_columns = [f'Close_{i+1}' for i in range(len(merged_df.columns))]
merged_df.columns = new_columns

# 변경된 열 이름 확인
print(merged_df.columns)
merged_df = pd.DataFrame(merged_df)

Index(['Close_1', 'Close_2', 'Close_3', 'Close_4', 'Close_5', 'Close_6',
       'Close_7', 'Close_8', 'Close_9', 'Close_10', 'Close_11', 'Close_12'],
      dtype='object')


In [34]:
# 열 이름에 숫자를 붙여 변경
new_columns = [f'Close_{i+1}' for i in range(len(amerged_df.columns))]
amerged_df.columns = new_columns

# 변경된 열 이름 확인
print(amerged_df.columns)
amerged_df = pd.DataFrame(amerged_df)

Index(['Close_1', 'Close_2', 'Close_3', 'Close_4', 'Close_5', 'Close_6',
       'Close_7', 'Close_8', 'Close_9', 'Close_10', 'Close_11', 'Close_12',
       'Close_13', 'Close_14'],
      dtype='object')


In [35]:
merged_df.to_csv('merged_df1.csv')

In [36]:
amerged_df.to_csv('amerged_df1.csv')

In [37]:
x.reset_index(inplace=True)
x['Date'] = pd.to_datetime(x['Date'])
x.set_index('Date')
x

,Date,KOSPI,USD/KRW,SP500
0,2022-01-04,2989.239990,1194.680054,4793.540039
1,2022-01-05,2953.969971,1196.500000,4700.580078
2,2022-01-06,2920.530029,1199.250000,4696.049805
3,2022-01-07,2954.889893,1205.780029,4677.029785
4,2022-01-10,2926.719971,1196.880005,4670.290039
...,...,...,...,...
468,2023-11-29,2519.810059,1287.530029,4550.580078
469,2023-11-30,2535.290039,1288.359985,4567.799805
470,2023-12-01,2505.010010,1296.660034,4594.629883
471,2023-12-04,2514.949951,1299.189941,4569.779785


In [38]:
merged_df.reset_index(inplace=True)
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df.set_index('Date')
merged_df = merged_df[(merged_df["Date"] >= start_date) & (merged_df["Date"] <= end_date)]
merged_df

,Date,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,Close_10,Close_11,Close_12
1,2022-01-04,56400,13000,42700,37600,9390,8470,8370,48000,79700,10400,6090,57300
2,2022-01-05,56600,13050,43000,37500,9480,8550,8390,47000,78900,10450,6260,55600
3,2022-01-06,55800,12850,43200,37300,9440,8480,8340,46800,76800,10350,6150,56000
4,2022-01-07,55700,13100,43450,37650,9490,8560,8340,49300,78800,10350,6390,55000
5,2022-01-10,57800,13950,45400,38550,9730,8830,8600,48250,80600,10650,6670,51100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,2023-11-21,54100,12720,41900,36950,8370,7130,10890,55800,58800,11700,8410,26000
465,2023-11-22,54500,12700,41800,36850,8360,7070,10940,55700,58500,11680,8450,26000
466,2023-11-23,54300,12710,42300,36950,8330,7110,10830,55700,58600,11670,8410,26000
467,2023-11-24,53700,12700,41750,36750,8350,7130,10830,54400,58000,11700,8380,25550


In [39]:
amerged_df.reset_index(inplace=True)
amerged_df['Date'] = pd.to_datetime(amerged_df['Date'])
amerged_df.set_index('Date')
amerged_df = amerged_df[(amerged_df["Date"] >= start_date) & (amerged_df["Date"] <= end_date)]
amerged_df

,Date,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,Close_10,Close_11,Close_12,Close_13,Close_14
1,2022-01-04,47.990002,63.590000,52.750000,167.830002,407.480011,58.770000,215.889999,62.349998,46.570000,98.320000,89.959999,104.260002,917.219971,222.460007
2,2022-01-05,47.180000,62.849998,52.290001,163.779999,398.630005,58.459999,212.389999,61.669998,46.430000,97.150002,89.309998,101.680000,891.770020,220.000000
3,2022-01-06,48.130001,64.910004,53.630001,165.520004,396.929993,60.049999,220.570007,63.980000,48.369999,98.919998,90.739998,103.419998,895.489990,219.750000
4,2022-01-07,49.180000,65.779999,54.770000,167.160004,397.510010,61.000000,223.729996,65.000000,48.799999,101.660004,91.739998,104.050003,892.270020,216.960007
5,2022-01-10,48.930000,66.029999,55.349998,167.320007,399.170013,61.360001,222.449997,65.290001,48.799999,101.199997,91.650002,104.129997,873.340027,211.970001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,2023-11-20,30.010000,45.259998,42.790001,153.289993,339.450012,37.630001,130.339996,32.040001,27.670000,69.949997,56.560001,79.699997,720.919983,249.970001
474,2023-11-21,29.660000,45.090000,42.599998,152.970001,334.970001,37.009998,127.970001,31.580000,27.360001,69.050003,55.990002,78.470001,724.640015,252.229996
475,2023-11-22,29.629999,45.020000,42.779999,153.330002,338.640015,36.830002,127.949997,31.490000,27.190001,69.879997,55.930000,78.489998,727.140015,253.720001
476,2023-11-24,29.730000,45.230000,42.919998,153.539993,339.149994,37.200001,128.699997,31.780001,27.250000,70.379997,56.430000,78.650002,730.099976,254.300003


In [40]:
# 주중, 주말 및 공휴일 설정
weekmask = 'Mon Tue Wed Thu Fri'
weekend = ['Sat', 'Sun']  # 주말 설정
today = datetime.now()
tomorrow = today + timedelta(1)
holidays = pd.to_datetime(['2022-01-01', '2023-11-30'])  # 공휴일 날짜 설정
today = today.strftime('%Y-%m-%d')

# CustomBusinessDay 객체 생성
custom_business_day = CustomBusinessDay(weekmask=weekmask, holidays=holidays, calendar=weekend)

# 주말과 공휴일을 포함한 모든 비즈니스 데이 날짜 생성
business_dates = pd.date_range(start=merged_df['Date'].min(), end=merged_df['Date'].max(), freq=custom_business_day)

# 주말과 공휴일을 포함한 모든 날짜로 데이터프레임 재생성
merged_df.set_index('Date', inplace=True)  # 인덱스 재설정
merged_df = merged_df.reindex(business_dates)

x.set_index('Date', inplace=True)
x = x.reindex(business_dates)

# 결측치를 이전 유효한 값으로 채우기
merged_df = merged_df.fillna(method='ffill')

x = x.fillna(method='bfill')

# 변경된 데이터프레임 출력
merged_df

,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,Close_10,Close_11,Close_12
2022-01-04,56400.0,13000.0,42700.0,37600.0,9390.0,8470.0,8370.0,48000.0,79700.0,10400.0,6090.0,57300.0
2022-01-05,56600.0,13050.0,43000.0,37500.0,9480.0,8550.0,8390.0,47000.0,78900.0,10450.0,6260.0,55600.0
2022-01-06,55800.0,12850.0,43200.0,37300.0,9440.0,8480.0,8340.0,46800.0,76800.0,10350.0,6150.0,56000.0
2022-01-07,55700.0,13100.0,43450.0,37650.0,9490.0,8560.0,8340.0,49300.0,78800.0,10350.0,6390.0,55000.0
2022-01-10,57800.0,13950.0,45400.0,38550.0,9730.0,8830.0,8600.0,48250.0,80600.0,10650.0,6670.0,51100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,54100.0,12720.0,41900.0,36950.0,8370.0,7130.0,10890.0,55800.0,58800.0,11700.0,8410.0,26000.0
2023-11-22,54500.0,12700.0,41800.0,36850.0,8360.0,7070.0,10940.0,55700.0,58500.0,11680.0,8450.0,26000.0
2023-11-23,54300.0,12710.0,42300.0,36950.0,8330.0,7110.0,10830.0,55700.0,58600.0,11670.0,8410.0,26000.0
2023-11-24,53700.0,12700.0,41750.0,36750.0,8350.0,7130.0,10830.0,54400.0,58000.0,11700.0,8380.0,25550.0


In [41]:
# 주말과 공휴일을 포함한 모든 날짜로 데이터프레임 재생성
amerged_df.set_index('Date', inplace=True)  # 인덱스 재설정
amerged_df = amerged_df.reindex(business_dates)

# 결측치를 이전 유효한 값으로 채우기
amerged_df = amerged_df.fillna(method='ffill')

# 변경된 데이터프레임 출력
amerged_df

,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,Close_10,Close_11,Close_12,Close_13,Close_14
2022-01-04,47.990002,63.590000,52.750000,167.830002,407.480011,58.770000,215.889999,62.349998,46.570000,98.320000,89.959999,104.260002,917.219971,222.460007
2022-01-05,47.180000,62.849998,52.290001,163.779999,398.630005,58.459999,212.389999,61.669998,46.430000,97.150002,89.309998,101.680000,891.770020,220.000000
2022-01-06,48.130001,64.910004,53.630001,165.520004,396.929993,60.049999,220.570007,63.980000,48.369999,98.919998,90.739998,103.419998,895.489990,219.750000
2022-01-07,49.180000,65.779999,54.770000,167.160004,397.510010,61.000000,223.729996,65.000000,48.799999,101.660004,91.739998,104.050003,892.270020,216.960007
2022-01-10,48.930000,66.029999,55.349998,167.320007,399.170013,61.360001,222.449997,65.290001,48.799999,101.199997,91.650002,104.129997,873.340027,211.970001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,29.660000,45.090000,42.599998,152.970001,334.970001,37.009998,127.970001,31.580000,27.360001,69.050003,55.990002,78.470001,724.640015,252.229996
2023-11-22,29.629999,45.020000,42.779999,153.330002,338.640015,36.830002,127.949997,31.490000,27.190001,69.879997,55.930000,78.489998,727.140015,253.720001
2023-11-23,29.629999,45.020000,42.779999,153.330002,338.640015,36.830002,127.949997,31.490000,27.190001,69.879997,55.930000,78.489998,727.140015,253.720001
2023-11-24,29.730000,45.230000,42.919998,153.539993,339.149994,37.200001,128.699997,31.780001,27.250000,70.379997,56.430000,78.650002,730.099976,254.300003


In [42]:
x

,KOSPI,USD/KRW,SP500
2022-01-04,2989.239990,1194.680054,4793.540039
2022-01-05,2953.969971,1196.500000,4700.580078
2022-01-06,2920.530029,1199.250000,4696.049805
2022-01-07,2954.889893,1205.780029,4677.029785
2022-01-10,2926.719971,1196.880005,4670.290039
...,...,...,...
2023-11-21,2510.419922,1288.150024,4538.189941
2023-11-22,2511.699951,1295.189941,4556.620117
2023-11-23,2514.959961,1298.790039,4559.339844
2023-11-24,2496.629883,1300.180054,4559.339844


In [43]:
merged_df.columns = ['KB', '우리', '하나', '신한', 'DGB', 'BNK', 'JB', '메리츠', '한국금융', '기업', '제주', '카카오']
merged_df

,KB,우리,하나,신한,DGB,BNK,JB,메리츠,한국금융,기업,제주,카카오
2022-01-04,56400.0,13000.0,42700.0,37600.0,9390.0,8470.0,8370.0,48000.0,79700.0,10400.0,6090.0,57300.0
2022-01-05,56600.0,13050.0,43000.0,37500.0,9480.0,8550.0,8390.0,47000.0,78900.0,10450.0,6260.0,55600.0
2022-01-06,55800.0,12850.0,43200.0,37300.0,9440.0,8480.0,8340.0,46800.0,76800.0,10350.0,6150.0,56000.0
2022-01-07,55700.0,13100.0,43450.0,37650.0,9490.0,8560.0,8340.0,49300.0,78800.0,10350.0,6390.0,55000.0
2022-01-10,57800.0,13950.0,45400.0,38550.0,9730.0,8830.0,8600.0,48250.0,80600.0,10650.0,6670.0,51100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,54100.0,12720.0,41900.0,36950.0,8370.0,7130.0,10890.0,55800.0,58800.0,11700.0,8410.0,26000.0
2023-11-22,54500.0,12700.0,41800.0,36850.0,8360.0,7070.0,10940.0,55700.0,58500.0,11680.0,8450.0,26000.0
2023-11-23,54300.0,12710.0,42300.0,36950.0,8330.0,7110.0,10830.0,55700.0,58600.0,11670.0,8410.0,26000.0
2023-11-24,53700.0,12700.0,41750.0,36750.0,8350.0,7130.0,10830.0,54400.0,58000.0,11700.0,8380.0,25550.0


In [44]:
amerged_df.columns = ['BAC', 'C', 'WFC', 'JPM', 'GS', 'USB', 'PNC', 'TFC', 'FITB', 'STT', 'SCHW', 'MS', 'BLK', 'V']
amerged_df

,BAC,C,WFC,JPM,GS,USB,PNC,TFC,FITB,STT,SCHW,MS,BLK,V
2022-01-04,47.990002,63.590000,52.750000,167.830002,407.480011,58.770000,215.889999,62.349998,46.570000,98.320000,89.959999,104.260002,917.219971,222.460007
2022-01-05,47.180000,62.849998,52.290001,163.779999,398.630005,58.459999,212.389999,61.669998,46.430000,97.150002,89.309998,101.680000,891.770020,220.000000
2022-01-06,48.130001,64.910004,53.630001,165.520004,396.929993,60.049999,220.570007,63.980000,48.369999,98.919998,90.739998,103.419998,895.489990,219.750000
2022-01-07,49.180000,65.779999,54.770000,167.160004,397.510010,61.000000,223.729996,65.000000,48.799999,101.660004,91.739998,104.050003,892.270020,216.960007
2022-01-10,48.930000,66.029999,55.349998,167.320007,399.170013,61.360001,222.449997,65.290001,48.799999,101.199997,91.650002,104.129997,873.340027,211.970001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,29.660000,45.090000,42.599998,152.970001,334.970001,37.009998,127.970001,31.580000,27.360001,69.050003,55.990002,78.470001,724.640015,252.229996
2023-11-22,29.629999,45.020000,42.779999,153.330002,338.640015,36.830002,127.949997,31.490000,27.190001,69.879997,55.930000,78.489998,727.140015,253.720001
2023-11-23,29.629999,45.020000,42.779999,153.330002,338.640015,36.830002,127.949997,31.490000,27.190001,69.879997,55.930000,78.489998,727.140015,253.720001
2023-11-24,29.730000,45.230000,42.919998,153.539993,339.149994,37.200001,128.699997,31.780001,27.250000,70.379997,56.430000,78.650002,730.099976,254.300003


In [45]:
merged_df.to_csv('merged_df.csv')

In [46]:
amerged_df.to_csv('amerged_df.csv')

### 데이터 합치기

In [47]:
# 시작 날짜와 끝나는 날짜 지정
start_date = "2022-01-04"
end_date = "2023-11-27"

In [48]:
# 시계열 생성
# df_c['폐업일자'] = pd.to_datetime(df_c['폐업일자'])
# df_c = df_c[(df_c["폐업일자"] >= start_date) & (df_c["폐업일자"] <= end_date)]

# 주말과 공휴일을 포함한 모든 날짜로 데이터프레임 재생성
# df_c.set_index('폐업일자', inplace=True)  # 인덱스 재설정
# df_c = df_c.reindex(business_dates)

# 결측치를 이전 유효한 값으로 채우기
# df_c = df_c.fillna(0)

# # 변경된 데이터프레임 출력
# df_c

In [49]:
# df_c = pd.DataFrame(df_c['일반음식점'])
# df_c = df_c.astype(int)
# df_c.info()

In [50]:
# df_c

In [51]:
x

,KOSPI,USD/KRW,SP500
2022-01-04,2989.239990,1194.680054,4793.540039
2022-01-05,2953.969971,1196.500000,4700.580078
2022-01-06,2920.530029,1199.250000,4696.049805
2022-01-07,2954.889893,1205.780029,4677.029785
2022-01-10,2926.719971,1196.880005,4670.290039
...,...,...,...
2023-11-21,2510.419922,1288.150024,4538.189941
2023-11-22,2511.699951,1295.189941,4556.620117
2023-11-23,2514.959961,1298.790039,4559.339844
2023-11-24,2496.629883,1300.180054,4559.339844


In [52]:
x1 = merged_df

In [53]:
x2 = amerged_df

# 결과

### 미국 연준은행 기준금리 인상 (1차) - 2022/03/16

0.25%

In [54]:
# 미 연준 금리인상(1차)
pre_period = ['2022-01-04', '2022-03-15']
post_period = ['2022-03-16', '2022-06-29']

In [55]:
kfirh1 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh1.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh1 = pd.DataFrame(kfirh1, columns=['종목,절대효과,상대효과,p-value'])
kfirh1.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh1.csv', index=False, encoding='ANSI')
kfirh1

,"종목,절대효과,상대효과,p-value"
0,"KB,-740.75,-1.27%,0.0"
1,"우리,7.29,0.05%,0.45"
2,"하나,-3033.96,-6.13%,0.0"
3,"신한,2983.83,7.85%,0.0"
4,"DGB,27.28,0.31%,0.11"
5,"BNK,335.56,4.56%,0.0"
6,"JB,462.55,5.74%,0.0"
7,"메리츠,8701.19,34.2%,0.0"
8,"한국금융,-6832.96,-8.79%,0.0"
9,"기업,108.46,1.0%,0.0"


### 미국 연준은행 기준금리 인상 (2차) - 2022/05/04

0.5%

In [56]:
# 미 연준 금리인상(2차)
pre_period = ['2022-01-04', '2022-05-03']
post_period = ['2022-05-04', '2022-06-15']

In [57]:
kfirh2 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh2.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh2 = pd.DataFrame(kfirh2, columns=['종목,절대효과,상대효과,p-value'])
kfirh2.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh2.csv', index=False, encoding='ANSI')
kfirh2

,"종목,절대효과,상대효과,p-value"
0,"KB,-516.55,-0.88%,0.02"
1,"우리,-48.82,-0.33%,0.24"
2,"하나,-973.64,-2.04%,0.0"
3,"신한,643.2,1.56%,0.0"
4,"DGB,-119.14,-1.36%,0.0"
5,"BNK,31.4,0.41%,0.1"
6,"JB,3.04,0.03%,0.44"
7,"메리츠,-3455.92,-9.84%,0.0"
8,"한국금융,-592.29,-0.85%,0.0"
9,"기업,-192.36,-1.7%,0.0"


### 미국 연준은행 기준금리 인상 (3차) - 2022/06/16

0.75%

In [58]:
# 미 연준 금리인상(3차)
pre_period = ['2022-01-04', '2022-06-15']
post_period = ['2022-06-16', '2022-07-25']

In [59]:
kfirh3 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh3.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh3 = pd.DataFrame(kfirh3, columns=['종목,절대효과,상대효과,p-value'])
kfirh3.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh3.csv', index=False, encoding='ANSI')
kfirh3

,"종목,절대효과,상대효과,p-value"
0,"KB,-4749.28,-9.02%,0.0"
1,"우리,-1838.88,-13.17%,0.0"
2,"하나,-4625.22,-10.76%,0.0"
3,"신한,-2381.96,-6.07%,0.0"
4,"DGB,-512.14,-6.26%,0.0"
5,"BNK,-478.01,-6.64%,0.0"
6,"JB,-867.47,-10.51%,0.0"
7,"메리츠,-1741.94,-6.28%,0.0"
8,"한국금융,-904.84,-1.46%,0.0"
9,"기업,-956.6,-9.09%,0.0"


### 미국 연준은행 기준금리 인상 (4차) - 2022/07/28

0.75%

In [60]:
# 미 연준 금리인상(4차)
pre_period = ['2022-01-04', '2022-07-27']
post_period = ['2022-07-28', '2022-09-19']

In [61]:
kfirh4 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh4.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh4 = pd.DataFrame(kfirh4, columns=['종목,절대효과,상대효과,p-value'])
kfirh4.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh4.csv', index=False, encoding='ANSI')
kfirh4

,"종목,절대효과,상대효과,p-value"
0,"KB,5006.48,11.25%,0.0"
1,"우리,112.44,0.95%,0.19"
2,"하나,5580.59,17.07%,0.0"
3,"신한,1860.68,5.52%,0.0"
4,"DGB,-210.22,-2.73%,0.0"
5,"BNK,-91.62,-1.35%,0.02"
6,"JB,196.32,2.71%,0.0"
7,"메리츠,-1550.62,-5.26%,0.0"
8,"한국금융,-1301.02,-2.17%,0.0"
9,"기업,552.04,6.12%,0.0"


### 미국 연준은행 기준금리 인상 (5차) - 2022/09/22

0.75%

In [62]:
# 미 연준 금리인상(5차)
pre_period = ['2022-01-04', '2022-09-21']
post_period = ['2022-09-22', '2022-10-31']

In [63]:
kfirh5 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh5.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh5 = pd.DataFrame(kfirh5, columns=['종목,절대효과,상대효과,p-value'])
kfirh5.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh5.csv', index=False, encoding='ANSI')
kfirh5

,"종목,절대효과,상대효과,p-value"
0,"KB,-809.83,-1.73%,0.04"
1,"우리,813.26,7.65%,0.0"
2,"하나,-195.95,-0.51%,0.31"
3,"신한,432.33,1.24%,0.04"
4,"DGB,-28.83,-0.41%,0.3"
5,"BNK,96.94,1.57%,0.02"
6,"JB,226.92,3.26%,0.0"
7,"메리츠,354.56,1.66%,0.28"
8,"한국금융,-1806.67,-3.54%,0.0"
9,"기업,412.22,4.31%,0.0"


### 미국 연준은행 기준금리 인상 (6차) - 2022/11/03

0.75%

In [64]:
# 미국의 소비자 물가지수 상승(6차)
pre_period = ['2022-01-04', '2022-11-02']
post_period = ['2022-11-03', "2022-12-12"]

In [65]:
kfirh6 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh6.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh6 = pd.DataFrame(kfirh6, columns=['종목,절대효과,상대효과,p-value'])
kfirh6.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh6.csv', index=False, encoding='ANSI')
kfirh6

,"종목,절대효과,상대효과,p-value"
0,"KB,1484.96,3.05%,0.0"
1,"우리,796.14,6.95%,0.0"
2,"하나,536.38,1.27%,0.09"
3,"신한,424.92,1.16%,0.06"
4,"DGB,218.33,3.05%,0.0"
5,"BNK,360.92,5.44%,0.0"
6,"JB,496.26,6.72%,0.0"
7,"메리츠,10980.18,50.75%,0.0"
8,"한국금융,-1166.52,-2.04%,0.02"
9,"기업,581.15,5.65%,0.0"


### 미국 연준은행 기준금리 인상 (7차) - 2022/12/15

0.5%

In [66]:
# 미국의 소비자 물가지수 상승(7차)
pre_period = ['2022-01-04', '2022-12-14']
post_period = ['2022-12-15', "2023-01-31"]

In [67]:
kfirh7 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh7.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh7 = pd.DataFrame(kfirh7, columns=['종목,절대효과,상대효과,p-value'])
kfirh7.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh7.csv', index=False, encoding='ANSI')
kfirh7

,"종목,절대효과,상대효과,p-value"
0,"KB,2314.54,4.44%,0.0"
1,"우리,-68.49,-0.54%,0.3"
2,"하나,2030.97,4.47%,0.0"
3,"신한,1839.13,4.84%,0.0"
4,"DGB,-319.72,-4.05%,0.0"
5,"BNK,-292.08,-4.0%,0.0"
6,"JB,1057.76,13.06%,0.0"
7,"메리츠,4040.76,10.53%,0.0"
8,"한국금융,-216.14,-0.37%,0.35"
9,"기업,-283.88,-2.65%,0.0"


### 미국 연준은행 기준금리 인상 (8차) - 2023/02/02

0.25%

In [68]:
# 미국 기준금리 인상(8차)
pre_period = ['2022-01-04', '2023-02-01']
post_period = ['2023-02-02', "2023-03-20"]

In [69]:
kfirh8 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh8.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh8 = pd.DataFrame(kfirh8, columns=['종목,절대효과,상대효과,p-value'])
kfirh8.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh8.csv', index=False, encoding='ANSI')
kfirh8

,"종목,절대효과,상대효과,p-value"
0,"KB,-7953.66,-13.38%,0.0"
1,"우리,-1633.53,-11.98%,0.0"
2,"하나,-8319.91,-15.59%,0.0"
3,"신한,-6582.79,-14.71%,0.0"
4,"DGB,-448.12,-5.53%,0.0"
5,"BNK,-772.72,-10.37%,0.0"
6,"JB,-1173.04,-11.13%,0.0"
7,"메리츠,1719.21,4.21%,0.01"
8,"한국금융,-1368.36,-2.23%,0.01"
9,"기업,-1229.42,-10.86%,0.0"


### SVB 파산 - 2023/03/10

In [70]:
# SVB 파산
pre_period = ['2022-01-04', '2023-03-09']
post_period = ['2023-03-10', "2023-06-29"]

In [71]:
kfirhs = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirhs.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirhs = pd.DataFrame(kfirhs, columns=['종목,절대효과,상대효과,p-value'])
kfirhs.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh_SVB.csv', index=False, encoding='ANSI')
kfirhs

,"종목,절대효과,상대효과,p-value"
0,"KB,-7316.78,-13.14%,0.0"
1,"우리,-2020.5,-14.78%,0.0"
2,"하나,-7000.15,-14.5%,0.0"
3,"신한,-7249.79,-17.14%,0.0"
4,"DGB,-1510.04,-17.67%,0.0"
5,"BNK,-878.23,-11.65%,0.0"
6,"JB,-1577.56,-15.5%,0.0"
7,"메리츠,-6518.45,-13.01%,0.0"
8,"한국금융,-8040.57,-12.79%,0.0"
9,"기업,-2249.04,-18.24%,0.0"


### 미국 연준은행 기준금리 인상 (9차) - 2023/03/23

0.25%

In [72]:
# 미국 연준의 금리인상(8차)
pre_period = ['2022-01-04', '2023-03-22']
post_period = ['2023-03-23', "2023-05-01"]

In [73]:
kfirh9 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh9.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh9 = pd.DataFrame(kfirh9, columns=['종목,절대효과,상대효과,p-value'])
kfirh9.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh9.csv', index=False, encoding='ANSI')
kfirh9

,"종목,절대효과,상대효과,p-value"
0,"KB,104.66,0.22%,0.41"
1,"우리,517.27,4.73%,0.0"
2,"하나,341.04,0.83%,0.24"
3,"신한,300.8,0.86%,0.18"
4,"DGB,-441.22,-6.01%,0.0"
5,"BNK,163.97,2.56%,0.0"
6,"JB,-419.18,-4.64%,0.0"
7,"메리츠,908.48,2.17%,0.1"
8,"한국금융,-2744.75,-4.74%,0.0"
9,"기업,-75.93,-0.75%,0.15"


### FPB(퍼스트리퍼블릭) 파산 - 2023/05/02

In [74]:
# FPB 파산
pre_period = ['2022-01-04', '2023-05-01']
post_period = ['2023-05-02', "2023-07-31"]

In [75]:
kfirhf = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirhf.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirhf = pd.DataFrame(kfirhf, columns=['종목,절대효과,상대효과,p-value'])
kfirhf.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh_FPB.csv', index=False, encoding='ANSI')
kfirhf

,"종목,절대효과,상대효과,p-value"
0,"KB,-2703.76,-5.29%,0.0"
1,"우리,-511.86,-4.15%,0.0"
2,"하나,-3401.49,-7.76%,0.0"
3,"신한,-495.81,-1.42%,0.04"
4,"DGB,-137.62,-1.89%,0.02"
5,"BNK,-173.46,-2.5%,0.0"
6,"JB,-352.43,-3.99%,0.0"
7,"메리츠,-5961.22,-11.76%,0.0"
8,"한국금융,-6207.46,-10.49%,0.0"
9,"기업,-411.01,-3.86%,0.0"


### 미국 연준은행 기준금리 인상 (10차) - 2023/05/04

0.25%

In [76]:
# 미 연준 금리인상(9차)
pre_period = ['2022-01-04', '2023-05-03']
post_period = ['2023-05-04', '2023-07-24']

In [77]:
kfirh10 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh10.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh10 = pd.DataFrame(kfirh10, columns=['종목,절대효과,상대효과,p-value'])
kfirh10.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh10.csv', index=False, encoding='ANSI')
kfirh10

,"종목,절대효과,상대효과,p-value"
0,"KB,-2537.48,-5.0%,0.0"
1,"우리,-423.17,-3.45%,0.0"
2,"하나,-3409.08,-7.77%,0.0"
3,"신한,-569.38,-1.62%,0.04"
4,"DGB,-120.45,-1.66%,0.03"
5,"BNK,-150.3,-2.17%,0.0"
6,"JB,-212.21,-2.44%,0.0"
7,"메리츠,-6498.18,-12.78%,0.0"
8,"한국금융,-5797.23,-9.81%,0.0"
9,"기업,-279.08,-2.66%,0.0"


### 미국 연준은행 기준금리 인상 (11차) - 2023/07/27

0.25%

In [78]:
# 미 연준 금리인상(11차)
pre_period = ['2022-01-04', '2023-07-26']
post_period = ['2023-07-27', '2023-11-23']

In [79]:
kfirh11 = []

for index, row in merged_df.iteritems():
    try:
        if row.sum() > 2:
            ci = pd.DataFrame(row).join(x, how='inner')
            ci = ci.join(x2, how = 'left')
        cib = CausalImpact(ci, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = cib.summary_data.to_dict()
        kfirh11.append([index + "," + str(round(summary['average'].get('abs_effect'),2)) + ',' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(cib.p_value,2))])
    except ValueError as e:
        print(e)

kfirh11 = pd.DataFrame(kfirh11, columns=['종목,절대효과,상대효과,p-value'])
kfirh11.to_csv('C:/Users/Edward/Desktop/PythonWorkspace/paper/firh11.csv', index=False, encoding='ANSI')
kfirh11

,"종목,절대효과,상대효과,p-value"
0,"KB,9282.77,20.77%,0.0"
1,"우리,775.18,6.85%,0.0"
2,"하나,2475.08,6.44%,0.0"
3,"신한,1581.16,4.65%,0.0"
4,"DGB,880.5,12.68%,0.0"
5,"BNK,512.09,8.03%,0.0"
6,"JB,1641.53,19.93%,0.0"
7,"메리츠,12763.73,32.02%,0.0"
8,"한국금융,6797.64,14.65%,0.0"
9,"기업,844.73,8.22%,0.0"
